In [6]:
!pip install -q yt_dlp deeplake langchain openai

In [7]:
pip install -q openai-whisper

In [8]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [9]:
import yt_dlp

def download_youtube(url):
  filename="video.mp4"
  ydl_opts = {
      'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
      'outtmpl': filename,
      'quiet': True,
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    result = ydl.extract_info(url, download=True)

url = "https://www.youtube.com/watch?v=aU-8BbPRqbo"
download_youtube(url)

In [42]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("video.mp4")

In [42]:
!pip install -q langchain-openai langchain-community langchain-core

In [39]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema.runnable import RunnablePassthrough

from langchain_text_splitters import RecursiveCharacterTextSplitter

content = result['text']
llm=ChatOpenAI(model="gpt-4o-mini")

In [14]:
from langchain.schema import Document
from langchain_community.document_loaders import TextLoader

with open('content.txt','w') as file:
  file.write(content)

loader = TextLoader('content.txt')
documents = loader.load()

In [20]:
# print(documents[0])
print(type(documents[0]))
print(len(documents))

<class 'langchain_core.documents.base.Document'>
1


In [21]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

splits = text_splitter.split_documents(documents)

In [22]:
print(len(splits))
print(type(splits[0]))

36
<class 'langchain_core.documents.base.Document'>


In [24]:
docs = [Document(page_content=t.page_content) for t in splits]
print(len(docs))

36


# **Summarization**

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
import warnings
warnings.filterwarnings("ignore")

template = hub.pull("rlm/map-prompt")
prompt = ChatPromptTemplate.from_messages(template.messages)

chain = prompt | llm | StrOutputParser()
print(chain.invoke(docs))

Based on the provided documents, the main themes can be summarized as follows:

1. **Entrepreneurship vs. Technical Skills**: The discussion emphasizes a common misconception known as the "entrepreneurial myth," which suggests that individuals who are skilled in a technical domain (like baking or teaching) automatically possess the skills needed to run a successful business. The importance of balancing technical skills with entrepreneurial and managerial skills is highlighted.

2. **The Triple Threat**: This theme revolves around the necessity for individuals in business to embody three distinct roles: the technician (the doer), the entrepreneur (the visionary), and the manager (the organizer). The success of a business relies on finding a balance among these roles.

3. **The Infancy Trap**: This concept describes the stage where a business owner is primarily working in the business (as a technician) rather than on it (as an entrepreneur or manager). The discussion underscores the impo

In [34]:
print(template[0])

prompt=PromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, template='The following is a set of documents:\n{docs}\nBased on this list of docs, please identify the main themes \nHelpful Answer:') additional_kwargs={}


# **Ask Any Question**

In [42]:
!pip install -q chromadb

In [38]:
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""

In [40]:
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {'context':retriever, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [41]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        break
    else:
        print("Bot:", chain.invoke(user_input))

User: tell me theme
Bot: The theme of the context revolves around the challenges and realities of entrepreneurship, particularly the balance between pursuing one's passion (like baking pies) and managing the various responsibilities that come with running a successful business. It highlights the importance of understanding business dynamics, such as marketing, finance, and leadership, while also addressing the personal desires of entrepreneurs to focus solely on their craft. The narrative emphasizes that to thrive as a business owner, one must engage with all aspects of the business rather than just the technical work they love.
User: q
